# Natural Language Process on Dataset

In [2]:
import os
import numpy as np
import pandas as pd
import nltk
nltk.download('reuters')
from nltk.corpus import reuters
from nltk.tokenize import MWETokenizer
from nltk import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import spacy
nlp = spacy.load("en_core_web_sm")
from collections import Counter

# nltk.download('stopwords')
# nltk.download('wordnet')

[nltk_data] Downloading package reuters to /root/nltk_data...


## Import Files

In [ ]:
food_protein =  pd.read_csv (r'C:\Users\gghan\OneDrive\Desktop\CapstoneProject\ProcessedData\food_protein.csv')

In [ ]:
food_protein.head()

,Unnamed: 0,fdc_id,data_type,description,food_category_id,protein_value,fat_value,carbohydrate_value
0,0,167518,sr_legacy_food,"Waffle, buttermilk, frozen, ready-to-heat, mic...",NaN,4.0,9.0,4.0
1,1,167532,sr_legacy_food,"Bread, white wheat",NaN,4.0,9.0,4.0
2,2,167537,sr_legacy_food,"Snacks, corn-based, extruded, chips, plain",NaN,2.7,8.8,4.0
3,3,167538,sr_legacy_food,"Snacks, corn-based, extruded, chips, barbecue-...",NaN,2.7,8.7,3.9
4,4,167539,sr_legacy_food,"Snacks, corn-based, extruded, cones, plain",NaN,3.5,8.6,4.1


In [ ]:
for food in food_protein['description']:

    print(food)

Waffle, buttermilk, frozen, ready-to-heat, microwaved
Bread, white wheat
Snacks, corn-based, extruded, chips, plain
Snacks, corn-based, extruded, chips, barbecue-flavor
Snacks, corn-based, extruded, cones, plain
Snacks, fruit leather, pieces
Snacks, fruit leather, rolls
Snacks, granola bars, hard, plain
Snacks, granola bars, hard, almond
Snacks, granola bars, soft, uncoated, raisin
Snacks, granola bars, soft, coated, milk chocolate coating, chocolate chip
Candies, honey-combed, with peanut butter
Snacks, granola bars, soft, coated, milk chocolate coating, peanut butter
Snacks, granola bars, soft, uncoated, peanut butter and chocolate chip
Snacks, popcorn, oil-popped, microwave, regular flavor, no trans fat
Snacks, popcorn, cakes
Snacks, popcorn, caramel-coated, with peanuts
Snacks, popcorn, caramel-coated, without peanuts
Snacks, potato chips, made from dried potatoes, reduced fat
Snacks, potato chips, made from dried potatoes, sour-cream and onion-flavor
Snacks, pretzels, hard, plain,

Lamb, leg, whole (shank and sirloin), separable lean only, trimmed to 1/4" fat, choice, raw
Lamb, leg, whole (shank and sirloin), separable lean only, trimmed to 1/4" fat, choice, cooked, roasted
Lamb, leg, shank half, separable lean and fat, trimmed to 1/4" fat, choice, raw
Lamb, leg, sirloin half, separable lean and fat, trimmed to 1/4" fat, choice, cooked, roasted
Lamb, leg, sirloin half, separable lean only, trimmed to 1/4" fat, choice, raw
Lamb, leg, sirloin half, separable lean only, trimmed to 1/4" fat, choice, cooked, roasted
Lamb, loin, separable lean and fat, trimmed to 1/4" fat, choice, raw
Lamb, loin, separable lean only, trimmed to 1/4" fat, choice, cooked, roasted
Lamb, rib, separable lean and fat, trimmed to 1/4" fat, choice, raw
Lamb, rib, separable lean and fat, trimmed to 1/4" fat, choice, cooked, broiled
Lamb, rib, separable lean and fat, trimmed to 1/4" fat, choice, cooked, roasted
Lamb, shoulder, whole (arm and blade), separable lean and fat, trimmed to 1/4" fat, c

## Tokenization Functions

In [ ]:
# Function for tokenization

def food_tokenize(document):
    word_list = []
    analyzed = nlp(document)
    # Loop through the word list
    for token in analyzed:
        if token.is_alpha and not token.is_stop:
            possible_add = token.lemma_.lower()       #Lemmatize and lowercase
            word_list.append(possible_add)
    return word_list

In [ ]:
def multiword_tokenize(doc, num_words):
    multiword_list = []
    num_words_lemma = []
    # Set stop words and lemmatizer
    stops = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    doc_lemma = ""
    for x in doc.split():
      x = x.replace(".", "")
      x = lemmatizer.lemmatize(x)
      doc_lemma = doc_lemma + " " + x
    for y in num_words:
      y = y.replace(".", "")
      y = lemmatizer.lemmatize(y)
      num_words_lemma.append(y)

    # Set up multiple word expressions
    mwe = MWETokenizer([num_words_lemma])
    doc_nocomma = doc_lemma.replace(',','')
    multi_analyzed = mwe.tokenize(doc_lemma.split())
    
    #Loop through the word list
    for multi_token in multi_analyzed:
        if multi_token.isalpha and multi_token not in stops:
            # Lowercase
            possible_add = multi_token.lower()
            # Lemmatize
            
            multiword_list.append(possible_add)
            
    return multiword_list
        

In [ ]:
# Testing Function

# doc = 'Parmesan cheese topping, fat free'
# food_name = doc.partition(',')[0]
# num_words = food_name.split()
# multiword_tokenize(doc, num_words)

## Food Protein Data NLP

### Tokenize food names

In [1]:
# Convert into a dataframe filled with lower cased form lists
food_dataframe = pd.DataFrame(columns = ['food_tokens'])
delimiter = ','
for food in food_protein['description']:
    food_name = food.partition(',')[0]       #Split food descriptions by commas
    num_words = food_name.split()        # Create a list of words using food descriptions
    
    # First word phrase is the name of the food, if the word phrase only have one word, tokenize it. Otherwise, use multword_tokenize function.
    if num_words == :                           
        food = delimiter.join(food)
        food_token = food_tokenize(food)
        food_dataframe = food_dataframe.append({'food_tokens': food_token}, ignore_index = True)
    else:
        multifood_token = multiword_tokenize(food, num_words)
        food_dataframe = food_dataframe.append({'food_tokens': multifood_token}, ignore_index = True)


NameError: ignored

In [ ]:
# Display the result of food tokens
for food in food_dataframe['food_tokens']:

    print(food)

['waffle', 'buttermilk', 'frozen', 'ready-to-heat', 'microwaved']
['bread', 'white', 'wheat']
['snack', 'corn-based', 'extruded', 'chip', 'plain']
['snack', 'corn-based', 'extruded', 'chip', 'barbecue-flavor']
['snack', 'corn-based', 'extruded', 'cone', 'plain']
['snack', 'fruit', 'leather', 'piece']
['snack', 'fruit', 'leather', 'roll']
['snack', 'granola', 'bar', 'hard', 'plain']
['snack', 'granola', 'bar', 'hard', 'almond']
['snack', 'granola', 'bar', 'soft', 'uncoated', 'raisin']
['snack', 'granola', 'bar', 'soft', 'coated', 'milk', 'chocolate', 'coating', 'chocolate', 'chip']
['candy', 'honey-combed', 'peanut', 'butter']
['snack', 'granola', 'bar', 'soft', 'coated', 'milk', 'chocolate', 'coating', 'peanut', 'butter']
['snack', 'granola', 'bar', 'soft', 'uncoated', 'peanut', 'butter', 'chocolate', 'chip']
['snack', 'popcorn', 'oil-popped', 'microwave', 'regular', 'flavor', 'trans', 'fat']
['snack', 'popcorn', 'cake']
['snack', 'popcorn', 'caramel-coated', 'peanut']
['snack', 'popco

['lupin', 'mature', 'seed', 'cooked', 'boiled', 'salt']
['mothbeans', 'mature', 'seed', 'cooked', 'boiled', 'salt']
['peanut', 'type', 'dry-roasted', 'without', 'salt']
['peanut', 'spanish', 'oil-roasted', 'without', 'salt']
['peanut', 'valencia', 'oil-roasted', 'without', 'salt']
['peanut', 'virginia', 'oil-roasted', 'without', 'salt']
['lamb', 'new', 'zealand', 'imported', 'frozen', 'loin', 'separable', 'lean', 'cooked', 'broiled']
['lamb', 'new', 'zealand', 'imported', 'frozen', 'shoulder', 'whole', '(arm', 'blade)', 'separable', 'lean', 'cooked', 'braised']
['veal', 'composite', 'trimmed', 'retail', 'cut', 'separable', 'lean', 'fat', 'raw']
['veal', 'composite', 'trimmed', 'retail', 'cut', 'separable', 'lean', 'fat', 'cooked']
['veal', 'composite', 'trimmed', 'retail', 'cut', 'separable', 'lean', 'raw']
['veal', 'leg', '(top', 'round)', 'separable', 'lean', 'fat', 'cooked', 'braised']
['veal', 'leg', '(top', 'round)', 'separable', 'lean', 'fat', 'cooked', 'pan-fried', 'breaded']
['

In [ ]:
print(f'Total number of Observations: {len(food_dataframe.index)}') 

Total number of Observations: 4771


### Update NLP results to the data

#### Select first tokens as food names

In [ ]:
processed_des = pd.DataFrame(columns = ['food_name'])
for food in food_dataframe['food_tokens']:
    processed_food = food[0]
    processed_des = processed_des.append({'food_name': processed_food}, ignore_index = True)

In [ ]:
for food in processed_des['food_name']:

    print(food)

waffle
bread
snack
snack
snack
snack
snack
snack
snack
snack
snack
candy
snack
snack
snack
snack
snack
snack
snack
snack
snack
snack
snack
snack
snack
candy
candy
baking_chocolate
candy
candy
candy
ice_cream
ice_cream
ice_cream
ice_cream
sherbet
candy
sweet_potatoes
deer_(venison)
restaurant
restaurant
restaurant
beverage
frozen_novelties
babyfood
sausage
pork_sausage_rice_links
salad_dressing
oil
beverage
beverage
pork
babyfood
babyfood
frozen_novelties
mayonnaise_dressing
pie_fillings
beverage
pudding
pudding
syrup
lemon
lemon_juice
lemon_juice_from_concentrate
prickly_pear
plum
plum
prune_juice
pummelo
raspberry
raspberry
raspberry
rhubarb
sapodilla
sapote
soursop
strawberry
tamarind
fruit_salad
watermelon
maraschino_cherry
pineapple
apricot
cherry
cherry
apple_juice
applesauce
applesauce
grapefruit_juice
apple_juice
abiyuch
rowal
guava_nectar
mango_nectar
tamarind_nectar
pomegranate_juice
nance
nance
naranjilla_(lulo)_pulp
horned_melon_(kiwano)
orange_juice
fruit_juice_smoothie
fru

mollusk
mollusk
mollusk
mollusk
mollusk
mollusk
mollusk
mollusk
fish
fish
fish
fish
fish
fish
fish
fish
fish
fish
fish
fish
fish
fish
fish
fish
fish
fish
fish
fish
fish
fish
fish
fish
fish
mollusk
mollusk
fish
lima_bean
lima_bean
lima_bean
lima_bean
mung_bean
mung_bean
noodle
mungo_beans
peanut
peanut
peanut
peanut
peanut
peanut_butter
peanut_butter
peanut_flour
meat_extender
sausage
soybean
tempeh
soy_flour
soy_flour
soy_flour
soy_protein_isolate
soy_sauce_made_from_soy_and_wheat_(shoyu)
soy_sauce_made_from_soy_(tamari)
soy_sauce_made_from_hydrolyzed_vegetable_protein
tofu
yardlong_beans
yardlong_beans
winged_bean
lentil
bean
bean
chickpea_flour_(besan)
hummus
tofu
tofu
peanut_butter
refried_beans
soybean
soybean
soy_protein_concentrate
soy_protein_isolate
tofu
tofu
tofu
yardlong_beans
lamb
lamb
lamb
lamb
lamb
lamb
lamb
lamb
lamb
lamb
lamb
lamb
lamb
lamb
lamb
lamb
lamb
lamb
lamb
lamb
lamb
lamb
lamb
lamb
lamb
lamb
lamb
lamb
lamb
lamb
lamb
lamb
lamb
lamb
game_meat
game_meat
game_meat
ga

#### Update original dataset

In [ ]:
food_name = processed_des['food_name']
food_protein = food_protein.join(food_name)

In [ ]:
food_protein.head()

,Unnamed: 0,fdc_id,data_type,description,food_category_id,protein_value,fat_value,carbohydrate_value,food_name
0,0,167518,sr_legacy_food,"Waffle, buttermilk, frozen, ready-to-heat, mic...",NaN,4.0,9.0,4.0,waffle
1,1,167532,sr_legacy_food,"Bread, white wheat",NaN,4.0,9.0,4.0,bread
2,2,167537,sr_legacy_food,"Snacks, corn-based, extruded, chips, plain",NaN,2.7,8.8,4.0,snack
3,3,167538,sr_legacy_food,"Snacks, corn-based, extruded, chips, barbecue-...",NaN,2.7,8.7,3.9,snack
4,4,167539,sr_legacy_food,"Snacks, corn-based, extruded, cones, plain",NaN,3.5,8.6,4.1,snack


#### Save to csv file

In [ ]:
food_protein.to_csv('NLP_food_protein.csv')

### Raw food number test

In [ ]:
df_raw = pd.DataFrame(columns = ['raw_food_tokens'])
for food in food_dataframe['food_tokens']:
    if 'raw' in food:
        df_raw = df_raw.append({'raw_food_tokens': food}, ignore_index = True)

In [ ]:
for food in df_raw['raw_food_tokens']:

    print(food)

['sausage', 'turkey', 'breakfast', 'link', 'mild', 'raw']
['cowpeas_(blackeyes)', 'immature', 'seed', 'raw']
['lamb', 'australian', 'imported', 'fresh', 'rib', 'chop/rack', 'roast', 'frenched', 'bone-in', 'separable', 'lean', 'trimmed', '1/8"', 'fat', 'raw']
['fish', 'pollock', 'alaska', 'raw', '(may', 'contain', 'additive', 'retain', 'moisture)']
['beef', 'ground', '90%', 'lean', 'meat', '/', '10%', 'fat', 'raw']
['lamb', 'australian', 'imported', 'fresh', 'shoulder', 'blade', 'separable', 'lean', 'trimmed', '1/8"', 'fat', 'raw']
['fish', 'tuna', 'fresh', 'bluefin', 'raw']
['beef', 'chuck', 'eye', 'steak', 'boneless', 'separable', 'lean', 'trimmed', '0"', 'fat', 'choice', 'raw']
['beef', 'round', 'eye', 'round', 'roast', 'boneless', 'separable', 'lean', 'trimmed', '0"', 'fat', 'grade', 'raw']
['lamb', 'ground', 'raw']
['beef', 'tenderloin', 'separable', 'lean', 'fat', 'trimmed', '1/8"', 'fat', 'prime', 'raw']
["nature's_kitchen", 'california', 'raw', 'almond']
['beef', 'loin', 'top', 

In [ ]:
print(f'raw food number: {len(df_raw.index)}') 

raw food number: 693


## Import food_nutrient_processed.csv

In [ ]:
food_nutrient =  pd.read_csv (r'C:\Users\gghan\OneDrive\Desktop\CapstoneProject\ProcessedData\food_nutrient_processed.csv')

In [ ]:
food_nutrient.head()

,Unnamed: 0,id,fdc_id,nutrient_id,description,amount,derivation_id,data_type,food_category_id,name,unit_name,nutrient_nbr,rank
0,0,1283674,167512,1003,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",5.88,46.0,sr_legacy_food,NaN,Protein,G,203.0,600.0
1,1,1283688,167513,1003,"Pillsbury, Cinnamon Rolls with Icing, refriger...",4.34,47.0,sr_legacy_food,NaN,Protein,G,203.0,600.0
2,2,1283712,167514,1003,"Kraft Foods, Shake N Bake Original Recipe, Coa...",6.10,1.0,sr_legacy_food,NaN,Protein,G,203.0,600.0
3,3,1283725,167515,1003,"George Weston Bakeries, Thomas English Muffins",8.00,47.0,sr_legacy_food,NaN,Protein,G,203.0,600.0
4,4,1283760,167516,1003,"Waffles, buttermilk, frozen, ready-to-heat",6.58,1.0,sr_legacy_food,NaN,Protein,G,203.0,600.0


## Food Nutrient Data NLP

In [ ]:
food_nutrient = food_nutrient.sample(frac = 0.005)

In [ ]:
# partial_food_nutrient =  pd.DataFrame(columns = ['fdc_id', 'data_type', 'description', 'food_category_id', 'protein_value', 'fat_value', 'carbohydrate_value'])
# fdc_list = food_nutrient['fdc_id'].tolist()
# len(fdc_list)
# for fdc_id in fdc_list:
        # if fdc_id not in food_protein['fdc_id']:
            
            # food_nutrient = food_nutrient.drop(food_nutrient[food_nutrient['fdc_id'] == fdc_id].index)

In [ ]:
food_nutrient.head()

,Unnamed: 0,id,fdc_id,nutrient_id,description,amount,derivation_id,data_type,food_category_id,name,unit_name,nutrient_nbr,rank
866731,866731,3420324,443143,1162,CAPRESE WITH CROSTINI,11.10,75.0,branded_food,NaN,"Vitamin C, total ascorbic acid",MG,401.0,6300.0
492371,492371,4495632,460280,1004,BALSAMIC VINAIGRETTE DRESSING & MARINADE,30.00,70.0,branded_food,NaN,Total lipid (fat),G,204.0,800.0
1857094,1857094,7905340,711676,1110,"PEPPERMINT PUFFS, PEPPERMINT",0.00,75.0,branded_food,NaN,"Vitamin D (D2 + D3), International Units",IU,324.0,8650.0
1766393,1766393,1319263,167948,1167,"Snacks, corn-based, extruded, onion-flavor",3.18,NaN,sr_legacy_food,NaN,Niacin,MG,406.0,6600.0
344640,344640,8176218,731404,1253,STRAWBERRY GRADE A MILK,10.00,70.0,branded_food,NaN,Cholesterol,MG,601.0,15700.0


In [ ]:
food_nutrient.count()

Unnamed: 0          9685
id                  9685
fdc_id              9685
nutrient_id         9685
description         9685
amount              9685
derivation_id       6539
data_type           9685
food_category_id    2270
name                9685
unit_name           9685
nutrient_nbr        9685
rank                9685
dtype: int64

### Tokenize food names

In [ ]:
# Convert into a dataframe filled with lower cased form lists
foodnutrient_dataframe = pd.DataFrame(columns = ['food_tokens'])
delimiter = ','
for food in food_nutrient['description']:
    food_name = food.partition(',')[0]       #Split food descriptions by commas
    num_words = food_name.split()        # Create a list of words using food descriptions
    
    # First word phrase is the name of the food, if the word phrase only have one word, tokenize it. Otherwise, use multword_tokenize function.
    if num_words == 1:                           
        food = delimiter.join(food)
        food_token = food_tokenize(food)
        foodnutrient_dataframe = foodnutrient_dataframe.append({'food_tokens': food_token}, ignore_index = True)
    else:
        multifood_token = multiword_tokenize(food, num_words)
        foodnutrient_dataframe = foodnutrient_dataframe.append({'food_tokens': multifood_token}, ignore_index = True)

In [ ]:
# Display the result of food tokens
for food in foodnutrient_dataframe['food_tokens']:

    print(food)

['caprese_with_crostini']
['balsamic_vinaigrette_dressing_&_marinade']
['peppermint_puffs', 'peppermint']
['snack', 'corn-based', 'extruded', 'onion-flavor']
['strawberry_grade_a_milk']
['pie', 'blueberry', 'prepared', 'recipe']
['bean', 'pink', 'mature', 'seed', 'cooked', 'boiled', 'without', 'salt']
['peanut_butter_chocolate_chip_chewy_granola_bars', 'peanut', 'butter', 'chocolate', 'chip']
['turkey', 'whole', 'back', 'meat', 'cooked', 'roasted']
['stewed_corned_beef', 'puerto', 'rican', 'style']
['large_duck_eggs']
["annie's_gluten_free_cocoa_&_vanilla_bunny_cookies"]
['veal_with_gravy']
['key_lime_sour_cocktail_mix']
['lamb', 'shoulder', 'arm', 'separable', 'lean', 'fat', 'trimmed', '1/8"', 'fat', 'cooked', 'broiled']
['pork', 'cured', 'ham', 'shank', 'bone-in', 'separable', 'lean', 'fat', 'heated', 'roasted']
['whoopie_pie_chocolate_cake_with_white_filling']
['korean_chili_sauce', 'spicy']
['herbs_de_provence_brine_mix']
['bread', 'white', 'prepared', 'recipe', 'made', 'low', 'fat

['beer_cake_mix']
['pretzel', 'hard', 'plain', 'lightly', 'salted']
['classic_fancy_cut_wax_beans', 'classic']
['turtle', 'cooked', 'n', 'cooking', 'method']
['beef', 'plate', 'steak', 'boneless', 'inside', 'skirt', 'separable', 'lean', 'trimmed', '0"', 'fat', 'grade', 'raw']
["cardini's", 'caesar', 'dressing', 'garlic', 'lemon']
['organic_apple_condiment']
['crab', 'baked', 'broiled', 'added', 'fat']
['snack', 'tortilla', 'chip', 'nacho-flavor', 'made', 'enriched', 'masa', 'flour']
['pasta', 'fresh-refrigerated', 'plain', 'cooked']
['watermelon_endurance_formula_thirst_quencher_powder', 'watermelon']
['cupcake_cake']
['fat_free_skim_milk']
['yogurt', 'greek', 'vanilla', 'lowfat']
['swirl_marshmallow_pop']
['pork', 'fresh', 'backfat', 'raw']
['fresh_frozen_broccoli_cuts']
["kellogg's_pop-tarts_chocolate_fudge_22oz"]
['bananas_and_pineapple', 'baby', 'food', 'strained']
['candy', 'mar', 'snackfood', 'u', 'twix', 'peanut', 'butter', 'cookie', 'bar']
['green_pea', 'cooked', 'restaurant']


['original_shredded_wheat', 'original']
['cracker', 'cheese', 'reduced', 'fat']
['kalamata_pitted_olives']
['digiorno_pizza', 'pepperoni', 'topping', 'thin', 'crispy', 'crust', 'frozen', 'baked']
['honey_mustard_dressing', 'fat', 'free']
['brown_bread']
['smoked_norwegian_atlantic_salmon']
['ribbed_cracklings_in_old_fashioned']
['cheddar_egg_stuffed_bagels']
['infant_formula', 'liquid', 'concentrate', 'made', 'water', 'nfs', '(similac', 'isomil', 'soy)']
['pulled_chicken']
['slush_frozen_drink']
['turkey_gravy_mix']
['milk', 'lowfat', 'fluid', '1%', 'milkfat', 'added', 'vitamin', 'a', 'vitamin', 'd']
['beef', 'brisket', 'flat', 'half', 'separable', 'lean', 'trimmed', '0"', 'fat', 'choice', 'cooked', 'braised']
['italian_gelato', 'chocolate']
['cooky', 'vanilla', 'sandwich', 'creme', 'filling', 'reduced', 'fat']
['game_meat', 'bison', 'shoulder', 'clod', 'separable', 'lean', 'trimmed', '0"', 'fat', 'raw']
['amish_collection_red_raspberry_jam']
['fish', 'cusk', 'raw']
['snack', 'vegetabl

In [ ]:
print(f'Total number of Observations: {len(foodnutrient_dataframe.index)}') 

Total number of Observations: 9685


### Update NLP results to the data

#### Select first tokens as food names

In [ ]:
processed_des_nutrient = pd.DataFrame(columns = ['food_name'])
for food in foodnutrient_dataframe['food_tokens']:
    processed_food = food[0]
    processed_des_nutrient = processed_des_nutrient.append({'food_name': processed_food}, ignore_index = True)

In [ ]:
fdc_list = food_nutrient['fdc_id'].tolist()
processed_des_nutrient['fdc_id'] = fdc_list
for food in processed_des_nutrient['food_name']:

    print(food)

caprese_with_crostini
balsamic_vinaigrette_dressing_&_marinade
peppermint_puffs
snack
strawberry_grade_a_milk
pie
bean
peanut_butter_chocolate_chip_chewy_granola_bars
turkey
stewed_corned_beef
large_duck_eggs
annie's_gluten_free_cocoa_&_vanilla_bunny_cookies
veal_with_gravy
key_lime_sour_cocktail_mix
lamb
pork
whoopie_pie_chocolate_cake_with_white_filling
korean_chili_sauce
herbs_de_provence_brine_mix
bread
kellogg's_special_k_protein_bars_chocolatey_brownie_7.95oz
oil
lamb
kellogg's_mini-wheats_cereal_bite_size_frosted_1oz
beef
fish_wrap_sandwich
kashi_chewy_bars_trail_mix_12oz
cashews_halves_&_pieces_made_with_sea_salt
beverage
apricot
frozen_fruit_juice_bar
creamy_hot_sauce
mother's_cookies_circus_animals_12oz
beef
ginger_chews_with_lychee
mayonnaise
coleslaw
beet_green
sapote
taco_bell
edible_chocolate_candle
fruit_on_the_go
pizza_sausage
coconut_melts_dark_chocolate
potato
oil
sliced_banana_&_strawberry_fruit_crisps
cherry
unsweetened_cold_pressed_coffee
couscous
dried_ginger
hemp

banquet_brown_n_serve_original_patty
pepper
macaroni_or_noodles_with_cheese_and_meat
mollusk
duck
meatball_mix
margarine
kale_puffs
salad_dressing
pork_jerky
beef
beverage
coffee
babyfood
2%_reduced_fat_milk
salad_dressing
sugar_free_chewing_gum
cereals_ready-to-eat
jams_and_preserves
mcdonald's
chocolate_milk
wild_pig
maranatha
potato
wheat_flour
biscuit
fish
cereal_(post_cocoa_pebbles)
rutabaga
organic_valley
swiss_preserves
whole_grain_bread
asparagus
chocolate_milk
agave_liquid_sweetener
pepper_jack_cheese
soup
ice_cream
cracker
doughnut
orange_juice
spice
salsa
beverage
original_bbq_sauce
sauce
tortilla_chip
annie's_organic_cinnabunnies_cereal
vinegar
spice
handcrafted_whoopie_pies
grapefruit_juice
fanta_grape_soda_bottle
popeyes
beverage
fettuccine_enriched_spaghetti_product
keebler_private_label_crackers_original_4oz_50ct
cake_or_cupcake
infant_formula
hormel_always_tender
pie
simply_roundy's
pretzel
sour_cream
annie's_organic_chewy_oatmeal_raisin_granola_bar
essentials_shake_dr

honey_ham_&_swiss_cheese_pinwheel_tray
beef_jerky
dried_cranberries
pretzel_chips
beef_&_bean_burritos
beet_juice
cheese_sandwich
peanut
himalayan_pink_salt
dips_and_spreads
graham_cracker_crust
cereal
pork
w.l._foods
pork_hash
sunflower_seed
restaurant
russet_potatoes
keebler_grahams_crackers_original_15oz
peter_pan_creamy_honey_roast_peanut_spread
pea
macaroni_or_noodles_with_cheese
popcorn
chocolate
blueberry_acai
infant_formula
bread
rice
milk_chocolate_miniatures_peanut_butter_cups
peach
pecan
au_jus_gravy_mix
ritz_potato_crisps_sea_salt_100_gr
rhubarb
beef
bittersweet_chocolate_ice_cream
broccoli
turkey_&_cream_havarti_cheese_baguette
baby_swiss_cheese
kettle_chips
whole_foods_market
soup
ham
spearmint
peanut_butter_and_jelly_sandwich
game_meat
chicken
mollusk
cream_cheese_creme_cake
crustacean
creole_seasoning
natto
infant_formula
chocolate_cookie
garlic_&_herb_macaroni_&_goat_cheese_pasta
granola_maple_pecan
mung_bean
apple
rice
coffee
cheeseburger
non-fat_plain_greek_yogurt
la

In [ ]:
processed_des_nutrient.head()

,food_name,fdc_id
0,caprese_with_crostini,443143
1,balsamic_vinaigrette_dressing_&_marinade,460280
2,peppermint_puffs,711676
3,snack,167948
4,strawberry_grade_a_milk,731404


In [ ]:
food_nutrient.reset_index(drop=True)

,Unnamed: 0,id,fdc_id,nutrient_id,description,amount,derivation_id,data_type,food_category_id,name,unit_name,nutrient_nbr,rank
0,866731,3420324,443143,1162,CAPRESE WITH CROSTINI,11.10,75.0,branded_food,NaN,"Vitamin C, total ascorbic acid",MG,401.0,6300.0
1,492371,4495632,460280,1004,BALSAMIC VINAIGRETTE DRESSING & MARINADE,30.00,70.0,branded_food,NaN,Total lipid (fat),G,204.0,800.0
2,1857094,7905340,711676,1110,"PEPPERMINT PUFFS, PEPPERMINT",0.00,75.0,branded_food,NaN,"Vitamin D (D2 + D3), International Units",IU,324.0,8650.0
3,1766393,1319263,167948,1167,"Snacks, corn-based, extruded, onion-flavor",3.18,NaN,sr_legacy_food,NaN,Niacin,MG,406.0,6600.0
4,344640,8176218,731404,1253,STRAWBERRY GRADE A MILK,10.00,70.0,branded_food,NaN,Cholesterol,MG,601.0,15700.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9680,809248,3420640,549435,1104,SPREAD & DRESSING,0.00,75.0,branded_food,NaN,"Vitamin A, IU",IU,318.0,7500.0
9681,804423,3250850,446765,1104,"BARRY'S GOURMET, BROWNIE BITES",0.00,75.0,branded_food,NaN,"Vitamin A, IU",IU,318.0,7500.0
9682,959271,13092913,1100859,1087,"Tortilla, whole wheat",244.00,NaN,survey_fndds_food,4208.0,"Calcium, Ca",MG,301.0,5300.0
9683,1122992,13084938,1100736,1166,Zwieback toast,0.23,NaN,survey_fndds_food,5202.0,Riboflavin,MG,405.0,6500.0


In [ ]:
processed_des_nutrient.astype(str)

,food_name,fdc_id
0,caprese_with_crostini,443143
1,balsamic_vinaigrette_dressing_&_marinade,460280
2,peppermint_puffs,711676
3,snack,167948
4,strawberry_grade_a_milk,731404
...,...,...
9680,spread_&_dressing,549435
9681,barry's_gourmet,446765
9682,tortilla,1100859
9683,zwieback_toast,1100736


#### Update original dataset

In [ ]:
food_name = processed_des_nutrient['food_name']
food_nutrient = food_nutrient.join(processed_des_nutrient.set_index('fdc_id'), on = 'fdc_id')

In [ ]:
food_nutrient.head()

,Unnamed: 0,id,fdc_id,nutrient_id,description,amount,derivation_id,data_type,food_category_id,name,unit_name,nutrient_nbr,rank,food_name
866731,866731,3420324,443143,1162,CAPRESE WITH CROSTINI,11.10,75.0,branded_food,NaN,"Vitamin C, total ascorbic acid",MG,401.0,6300.0,caprese_with_crostini
492371,492371,4495632,460280,1004,BALSAMIC VINAIGRETTE DRESSING & MARINADE,30.00,70.0,branded_food,NaN,Total lipid (fat),G,204.0,800.0,balsamic_vinaigrette_dressing_&_marinade
1857094,1857094,7905340,711676,1110,"PEPPERMINT PUFFS, PEPPERMINT",0.00,75.0,branded_food,NaN,"Vitamin D (D2 + D3), International Units",IU,324.0,8650.0,peppermint_puffs
1766393,1766393,1319263,167948,1167,"Snacks, corn-based, extruded, onion-flavor",3.18,NaN,sr_legacy_food,NaN,Niacin,MG,406.0,6600.0,snack
1766393,1766393,1319263,167948,1167,"Snacks, corn-based, extruded, onion-flavor",3.18,NaN,sr_legacy_food,NaN,Niacin,MG,406.0,6600.0,snack


#### Save to csv file

In [ ]:
food_protein.to_csv('NLP_food_protein.csv')